In [1]:
import sys, os, time
import numpy as np
import gc
import pickle
%load_ext autoreload
%autoreload 2
%matplotlib inline
#%matplotlib tk
import matplotlib.pyplot as plt
import networkx as nx
import tnt_util as util
import inspect
from structures import xset, adict, idict, tdict, tset, tlist, Transactionable, AbortTransaction, Resilient, ResDict

In [2]:
a = ResDict()

In [3]:
a.a = 10
a.b = 'test'
print(a)

OrderedDict([('a', 10), ('b', 'test')])


In [4]:
print(type(a).__name__)

ResDict


In [5]:
parents = inspect.getmro(a.__class__)

In [6]:
for p in parents:
    print(inspect.getsource(p))

class ResDict(Resilient):
	def __init__(self, *args, **kwargs):
		self._data = OrderedDict(*args, **kwargs)
		
	def __setattr__(self, key, item):
		if key == '_data':
			super().__setattr__(key, item)
		else:
			self._data[key] = item
		
	def __getattr__(self, item):
		if item == '_data':
			return super().__getattr__(item)
		return self._data[item]
	
	def __delattr__(self, item):
		assert item != '_data', 'cannot delete "_data"'
		del self._data[item]
		
	def __repr__(self):
		return repr(self._data)
		
	def __str__(self):
		return str(self._data)

class Resilient(object):
	
	def meta(self):
		return inspect.getsource(type(self))
	
	def state_dict(self):
		raise NotImplementedError
	
	def load_state_dict(self, state):
		raise NotImplementedError
	
	def copy(self):
		raise NotImplementedError



TypeError: <module 'builtins' (built-in)> is a built-in class

In [7]:
import sys
sys.modules.keys()

dict_keys(['sys', 'builtins', '_frozen_importlib', '_imp', '_thread', '_warnings', '_weakref', 'zipimport', '_frozen_importlib_external', '_io', 'marshal', 'posix', 'encodings', 'codecs', '_codecs', 'encodings.aliases', 'encodings.utf_8', '_signal', '__main__', 'encodings.latin_1', 'io', 'abc', '_abc', '_bootlocale', '_locale', 'site', 'os', 'stat', '_stat', 'posixpath', 'genericpath', 'os.path', '_collections_abc', '_sitebuiltins', 'types', 'importlib', 'importlib._bootstrap', 'importlib._bootstrap_external', 'warnings', 'importlib.util', 'importlib.abc', 'importlib.machinery', 'contextlib', 'collections', 'operator', '_operator', 'keyword', 'heapq', '_heapq', 'itertools', 'reprlib', '_collections', 'functools', '_functools', 'mpl_toolkits', 'sphinxcontrib', 'zope', 'encodings.cp437', 'runpy', 'pkgutil', 'weakref', '_weakrefset', 'ipykernel', 'ipykernel._version', 'ipykernel.connect', '__future__', 'json', 'json.decoder', 're', 'enum', 'sre_compile', '_sre', 'sre_parse', 'sre_constant

In [10]:
a = adict()
a['test'] = 'asdf'
a['b'] = 10
a['x'] = [3,3]
a.m = 100 - a.b
a

{'test':'asdf', 'b':10, 'x':[3, 3], 'm':90}

In [11]:
def makeI():
    b = idict()
    b['test'] = 'asdf'
    b['b'] = 10
    b['x'] = [3,3]
    print(b, list(get_table()))
    del b
    print(list(get_table()))
makeI()

idict[0](test:asdf, b:10, x:[3, 3]) [0]
[]


In [12]:
a = tdict(t=10)
a.ls = tlist()
a

{'t':10, 'ls':[]}

In [13]:
a.begin()
a.in_transaction(), a.ls.in_transaction()

(True, True)

In [7]:
a.ls.extend(range(10))
a

{'t':10, 'ls':[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]}

In [8]:
a.v = 23.3
a

{'t':10, 'ls':[0, 1, 2, 3, 4, 5, 6, 7, 8, 9], 'v':23.3}

In [9]:
a.abort()
a

{'t':10, 'ls':[]}

In [48]:
a = tlist()
a.append(6)
a

[6]

In [49]:
a.append(tlist())
a

[6, []]

In [50]:
with a:
    a[-1].append('all')
    a.extend([1,3,4])
    print(a, len(a))
    raise AbortTransaction
a

[6, ['all'], 1, 3, 4] 5


[6, []]

In [21]:
a.begin()

[6, []]


In [22]:
from itertools import chain

In [23]:
x = [1,2]
y = [3,4]

In [26]:
list(chain(*[iter(i) for i in [x,y]]))

[1, 2, 3, 4]

In [10]:
a.begin()
a[0].append('all')
a.extend([1,3,4])
print(a, len(a))
a.abort()

[['all', 'all'], 1, 3, 4] 4


In [11]:
a

[['all', 'all']]

In [4]:
a = tdict()
a['test'] = 'asdf'
a.c = 3.4
a['b'] = 10
a['x'] = [3,3]
print('c' in a)
del a.b
a

True


{'c':3.4, 'test':'asdf', 'x':[3, 3]}

In [7]:
print(a)
a.begin()
print(a)
a['dd'] = 10000
print(a)
del a.x
print(a)
print('x' in a)
print(a)
print('dd' in a)
print(a)
a.abort()
print(a)
print('x' in a)
print('dd' in a)
a

adict(c:3.4, test:asdf, x:[3, 3])
adict(c:3.4, test:asdf, x:[3, 3])
adict(c:3.4, dd:10000, test:asdf, x:[3, 3])
adict(c:3.4, dd:10000, test:asdf)
False
adict(c:3.4, dd:10000, test:asdf)
True
adict(c:3.4, dd:10000, test:asdf)
adict(c:3.4, test:asdf, x:[3, 3])
True
False


{'c':3.4, 'test':'asdf', 'x':[3, 3]}

In [ ]:
class tlist(transactionable, list):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self._transaction = None
        
    def __getattribute__(self, key):
        try:
            return super().__getattribute__('_transaction').__getattribute__(key)
        except:
            return super().__getattribute__(key)
    
    def begin(self):
        self._transaction = self.copy()
        
        for item in self:
            if isinstance(item, transactionable):
                item.begin()
        
    def in_transaction(self):
        return self._transaction is not None
    def commit(self):
        
        for item in self:
            if isinstance(item, transactionable):
                item.commit()
        
        changes = self._transaction
        self._transaction = None
        if self._transaction is not None:
            self.clear()
            self.extend(changes)
    def abort(self):
        for item in self:
            if isinstance(item, transactionable):
                item.abort()
        self._transaction = None

In [48]:
class __Empty_Transaction(object):
    pass

_reserved_attributes = {'begin', 'in_transaction', 'commit', 'abort', '__class__', '_Transactionable__enter__', '_Transactionable__exit__', 
                   '_original', '_transaction', '_iteration_fns', '_original_type_name'}
class Transactionable(object):
    def __init__(self, original, iterations=None):
        if iterations is None:
            iterations = [lambda x: x.__dict__.items()]
        self._iteration_fns = iterations
        
        self._original_type_name = type(original).__name__
        
        assert original is not None, 'Cant create transactions for None'
        #global __reserved_attributes
        for key in _reserved_attributes:
            try:
                original.__getattribute__(key)
            except AttributeError:
                pass
            else:
                print('WARNING: Transactionable will make attribute {}.{} unavailable'.format(self._original_type_name, key))
        
        self._original = original
        self._transaction = None #__Empty_Transaction
        
        self.__class__ = original.__class__
        
        
    def __getattribute__(self, key):
        if key in _reserved_attributes:
            return super().__getattribute__(key)
        try:
            return self._transaction.__getattribute__(key)
        except:
            return self._original.__getattribute__(key)
        
    def __setattr__(self, key, value):
        if key in _reserved_attributes:
            return super().__setattr__(key, value)
        try:
            return self._transaction.__setattr__(key, value)
        except:
            return self._original.__setattr__(key, value)
        
    def begin(self):
        if self.in_transaction():
            self.abort()
        self._transaction = self._original.copy()
        
        for itr_fn in self._iteration_fns:
            for vals in itr_fn(self._transaction):
                if isinstance(vals, Transactionable):
                    vals.begin()
                else:
                    try:
                        for v in vals:
                            if isinstance(v, Transactionable):
                                v.begin()
                    except AttributeError:
                        pass
        
    def in_transaction(self):
        return self._transaction is not None
    def commit(self):
        if not self.in_transaction():
            return
        for itr_fn in self._iteration_fns:
            for vals in itr_fn(self._transaction):
                if isinstance(vals, Transactionable):
                    vals.commit()
                else:
                    try:
                        for v in vals:
                            if isinstance(v, Transactionable):
                                v.commit()
                    except AttributeError:
                        pass
        
        self._original = self._transaction
        self._transaction = None
    def abort(self):
        if not self.in_transaction():
            return
        for itr_fn in self._iteration_fns:
            for vals in itr_fn(self._transaction):
                if isinstance(vals, Transactionable):
                    vals.abort()
                else:
                    try:
                        for v in vals:
                            if isinstance(v, Transactionable):
                                v.abort()
                    except AttributeError:
                        pass
        
        self._transaction = None
    
#     def __str__(self):
#         if self._transaction is not None:
#             return str(self._transaction)
#         return str(self._original)
    
#     def __repr__(self):
#         if self._transaction is not None:
#             return repr(self._transaction)
#         return repr(self._original)
    
    def __repr__(self):
        return 'Failure'
    
    def __enter__(self):
        self.begin()
    def __exit__(self, type, *args):
        if type is None:
            self.commit()
        else:
            self.abort()

In [9]:
from wrapt import ObjectProxy
from itertools import chain

In [89]:
class ToggleObjects(ObjectProxy):
    def __init__(self, obj):
        super().__init__(obj)
        self._self_o1 = obj
        self._self_o2 = obj.copy()
        self._self_ind = True
    def switch(self):
        if self._self_ind:
            self.__wrapped__ = self._self_o2
        else:
            self.__wrapped__ = self._self_o1
        self._self_ind = not self._self_ind
    def __repr__(self):
        return self.__wrapped__.__repr__()
    


In [19]:
class Transactionable(ObjectProxy):
    def __init__(self, original, iterations=None):
        
        super().__init__(original)
        
        if iterations is None:
            iterations = [lambda x: chain(x.__dict__.items())]
        self._self_iteration_fns = iterations if isinstance(iterations, list) else [iterations]
        
        
        assert original is not None, 'Cant create transactions for None'
        
        self._self_original = original
        self._self_transaction = None
        
    def begin(self):
        if self.in_transaction():
            self.abort()
        self._self_transaction = self._self_original.copy()
        self.__wrapped__ = self._self_transaction
        
        for val in chain(itr_fn(self._self_transaction) for itr_fn in self._self_iteration_fns):
            if isinstance(val, Transactionable):
                val.begin()
        
    def in_transaction(self):
        return self._self_transaction is not None
    def commit(self):
        if not self.in_transaction():
            return
        for val in chain(itr_fn(self._self_transaction) for itr_fn in self._self_iteration_fns):
            if isinstance(val, Transactionable):
                val.commit()
        
        self._self_original = self._self_transaction
        
        self._self_transaction = None
        self.__wrapped__ = self._self_original
        
    def abort(self):
        if not self.in_transaction():
            return
        for val in chain(itr_fn(self._self_transaction) for itr_fn in self._self_iteration_fns):
            if isinstance(val, Transactionable):
                val.abort()
        
        self._self_transaction = None
        self.__wrapped__ = self._self_original
    
    def __repr__(self):
        return self.__wrapped__.__repr__()
    def __str__(self):
        return self.__wrapped__.__str__()
    
    def __enter__(self):
        self.begin()
    def __exit__(self, type, *args):
        if type is None:
            self.commit()
        else:
            self.abort()

In [20]:
class Test(object):
    x = 10
    _original = -5
    
    def copy(self):
        copy = Test()
        copy.__dict__ = self.__dict__.copy()
        return copy
    
    def __len__(self):
        return len(self.__dict__)
    
    def __str__(self):
        return str(self.__dict__)
    def __repr__(self):
        return 'Test({})'.format(repr(self.__dict__))

In [31]:
x = Test()#[1,2]
a = Transactionable(x)
a.test = 10
a

Test({'test': 10})

In [36]:
a.begin()

In [37]:
a.x = 2304
del a.test
a

Test({'x': 2304})

In [34]:
a.abort()
a

Test({'test': 10})

In [38]:
a.commit()
a

Test({'x': 2304})

In [49]:
class Test(object):
    x = 10
    _original = -5
    
    def copy(self):
        copy = Test()
        copy.__dict__ = self.__dict__.copy()
        return copy
    
    def __len__(self):
        return 10
    
    def __str__(self):
        return 'Success'
        return str(self.__dict__)
    def __repr__(self):
        return 'Success'
        return 'Test({})'.format(repr(self.__dict__))

In [10]:
with a:
    a.x = 10
a

None (None, None)


{'test':'asdf', 'b':10, 'x':[3, 3]}

In [1]:
a = slice(10,20,3)

In [3]:
a.start

10

In [11]:
a._transaction

In [22]:
a.m = 'dsf'

In [23]:
a

{'test':'asdf', 'b':10, 'x':[3, 3], 'm':'dsf'}

In [25]:
a._transaction

In [30]:
a = _DeleteItemFlag

In [33]:
a is _DeleteItemFlag

False